In [2]:
!pip install tsai fastai torch pandas numpy matplotlib seaborn

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0

In [1]:
import pandas as pd
import os

# Load the dataset
file_path = "/content/API-Call-Dataset.csv"
df = pd.read_csv(file_path)

# Display the initial few rows and columns for verification
print("Initial Data:")
print(df.head())
print("\nColumn Names:", df.columns)

# Define the column names
api_column = "API Code"

# Count the number of calls for each API
api_call_counts = df[api_column].value_counts().nlargest(4)
top_apis = api_call_counts.index

print("\nTop 4 APIs based on the number of calls:")
print(api_call_counts)

# Extract the directory from the original file path
output_directory = os.path.dirname(file_path)

# Filter and save each top API's data into separate CSV files in the same directory
for api in top_apis:
    output_path = os.path.join(output_directory, f"{api}_data.csv")
    api_df = df[df[api_column] == api]
    api_df.to_csv(output_path, index=False)
    print(f"Data for {api} saved to {output_path}")

Initial Data:
  API Code      Time of Call
0       A1  01-02-2025 00:00
1       A3  01-02-2025 00:17
2       A3  01-02-2025 00:19
3       A8  01-02-2025 00:37
4       A8  01-02-2025 00:47

Column Names: Index(['API Code', 'Time of Call'], dtype='object')

Top 4 APIs based on the number of calls:
API Code
A9    2451
A2    2438
A7    2410
A4    2402
Name: count, dtype: int64
Data for A9 saved to /content/A9_data.csv
Data for A2 saved to /content/A2_data.csv
Data for A7 saved to /content/A7_data.csv
Data for A4 saved to /content/A4_data.csv


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tsai.all import *
from fastai.tabular.all import *
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

In [4]:
def get_models(c_in=2, c_out=1, seq_len=24):

    lstm = nn.Sequential(
        nn.LSTM(input_size=c_in, hidden_size=64, num_layers=2, batch_first=True),
        nn.Linear(64, c_out)
    )

    gru = nn.Sequential(
        nn.GRU(input_size=c_in, hidden_size=64, num_layers=2, batch_first=True),
        nn.Linear(64, c_out)
    )

    mlp = nn.Sequential(
        nn.Flatten(),
        nn.Linear(seq_len * c_in, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, c_out)
    )

    fcn = FCN(c_in=c_in, c_out=c_out)
    resnet = ResNet(c_in=c_in, c_out=c_out)

    return {
        'LSTM': lstm,
        'GRU': gru,
        'MLP': mlp,
        'FCN': fcn,
        'ResNet': resnet
    }

In [5]:
def model_forward(model, x, model_name):

    if model_name in ['LSTM', 'GRU']:
        x = x.transpose(1, 2)
        if isinstance(model, nn.Sequential):
            output, _ = model[0](x)
            return model[1](output[:, -1, :])
    elif model_name == 'MLP':

        return model(x)
    else:

        return model(x)

In [6]:
def prepare_data(df, seq_length=24, split_ratio=0.8):

    df = df.sort_values('Time of Call')

    time_diffs = df['Time of Call'].diff().dt.total_seconds()
    time_diffs = time_diffs.fillna(time_diffs.mean())

    time_diffs = (time_diffs - time_diffs.mean()) / time_diffs.std()

    X, y = [], []
    for i in range(len(df) - seq_length):
        sequence = np.column_stack([
            time_diffs.iloc[i:i+seq_length],
            np.ones(seq_length)
        ])
        X.append(sequence)
        y.append([1])

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)

    X = X.transpose(0, 2, 1)

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    train_size = int(len(X) * split_ratio)
    splits = (list(range(train_size)), list(range(train_size, len(X))))

    tfms = [None, [TSRegression()]]
    dsets = TSDatasets(X, y, splits=splits, tfms=tfms)

    batch_size = min(64, len(X))
    dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, batch_size=batch_size)

    return dls, df['Time of Call'].iloc[-1]

In [8]:
def main():
    original_file = "API-Call-Dataset.csv"
    df_original = pd.read_csv(original_file)

    api_counts = df_original['API Code'].value_counts().nlargest(4)
    top_apis = api_counts.index

    last_timestamps = {}

    for api in top_apis:
        print(f"\nProcessing {api}...")

        file_path = f"{api}_data.csv"
        df = pd.read_csv(file_path)

        df['Time of Call'] = pd.to_datetime(df['Time of Call'], format='%d-%m-%Y %H:%M')

        dls, last_timestamp = prepare_data(df)
        last_timestamps[api] = last_timestamp

        models = get_models()

        for model_name, model in models.items():
            print(f"Training {model_name}...")

            class ModelWrapper(nn.Module):
                def __init__(self, base_model, model_name):
                    super().__init__()
                    self.base_model = base_model
                    self.model_name = model_name

                def forward(self, x):
                    return model_forward(self.base_model, x, self.model_name)

            wrapped_model = ModelWrapper(model, model_name)
            learn = Learner(dls, wrapped_model, metrics=[rmse])

            learn.fit_one_cycle(10, lr_max=1e-3)

    print("\nLast timestamps for each API:")
    for api, timestamp in last_timestamps.items():
        print(f"{api}: {timestamp}")

    return last_timestamps

if __name__ == "__main__":
    last_timestamps = main()


Processing A9...
Training LSTM...


epoch,train_loss,valid_loss,_rmse,time
0,0.772932,0.645551,0.803462,00:01
1,0.404186,0.018955,0.137676,00:01
2,0.185979,0.000040,0.006306,00:01
3,0.093260,0.000031,0.005605,00:01
4,0.048732,0.000025,0.004997,00:01
5,0.025992,0.000021,0.004551,00:01
6,0.014014,0.000018,0.004292,00:01
7,0.007601,0.000017,0.004092,00:01
8,0.004137,0.000016,0.003943,00:01
9,0.002258,0.000015,0.003933,00:01


Training GRU...


epoch,train_loss,valid_loss,_rmse,time
0,0.775731,0.462476,0.680056,00:01
1,0.331836,0.010873,0.104276,00:01
2,0.152538,0.000084,0.009182,00:01
3,0.076484,0.000010,0.003168,00:01
4,0.039960,0.000005,0.002322,00:01
5,0.021308,0.000004,0.002046,00:01
6,0.011484,0.000004,0.001894,00:01
7,0.006224,0.000003,0.001832,00:01
8,0.003385,0.000003,0.001813,00:01
9,0.001844,0.000003,0.001811,00:01


Training MLP...


...

Training FCN...


epoch,train_loss,valid_loss,_rmse,time
0,0.414246,0.263163,0.512995,00:02
1,0.164189,0.003026,0.055010,00:03
2,0.077036,0.000591,0.024319,00:02
3,0.039445,0.000945,0.030746,00:02
4,0.021149,0.000756,0.027496,00:02
5,0.011613,0.001445,0.038014,00:03
6,0.006454,0.000412,0.020290,00:03
7,0.003644,0.000326,0.018067,00:02
8,0.002071,0.000574,0.023957,00:02
9,0.001204,0.000363,0.019060,00:02


Training ResNet...


epoch,train_loss,valid_loss,_rmse,time
0,0.875658,0.778591,0.882378,00:05
1,0.321951,0.009011,0.094927,00:04
2,0.150158,0.002807,0.052984,00:04
3,0.076834,0.002776,0.052686,00:06
4,0.041090,0.002033,0.045087,00:04
5,0.022524,0.001685,0.041054,00:04
6,0.012528,0.001537,0.039208,00:06
7,0.007062,0.001492,0.038624,00:04
8,0.004044,0.001997,0.044684,00:04
9,0.002355,0.001697,0.041192,00:05



Last timestamps for each API:
A9: 2025-07-06 09:06:00
A2: 2025-07-06 09:45:00
A7: 2025-07-06 07:55:00
A4: 2025-07-06 10:22:00
